In [229]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import imutils
import time

from keras.preprocessing.image import load_img,img_to_array

from keras.applications.resnet50 import ResNet50
from keras.models import Sequential
from keras.layers import Dense,Dropout

os.environ['KMP_DUPLICATE_LIB_OK']='True' 

model_resnet50 = ResNet50(input_shape=(224,224,3),weights='imagenet')

for layer in model_resnet50.layers:
    layer.trainable=False

model1 = Sequential()
model1.add(model_resnet50)
model1.add(Dense(512,activation='relu',name='Dense1'))
model1.add(Dropout(0.3))
model1.add(Dense(2,activation='softmax',name='prediction'))

for layer in model_resnet50.layers:
    layer.trainable=True

#model1.load_weights("/Users/syedshakeeb/Desktop/savedmodels/weights.best.resnet50.hdf5")

In [3]:
import numpy as np
import cv2
import os
from keras.preprocessing.image import load_img,img_to_array
import time
from PIL import Image

In [16]:
def sliding_window(image, stepSize, windowSize):
    for y in range(0,image.shape[0],stepSize):
        for x in range(0,image.shape[1],stepSize):
            yield(x,y,image[y:y+windowSize[1],x:x+windowSize[0]])

In [73]:
for i in os.listdir("/Users/syedshakeeb/Downloads/baza_slika/"):
    if(i!='.DS_Store'):
        for j in os.listdir("/Users/syedshakeeb/Downloads/baza_slika/" + i + "/"):
            if(j!=".DS_Store"):
                image_file = cv2.imread("/Users/syedshakeeb/Downloads/baza_slika/" + i + "/" + j)
                while True:
                    for (x, y, window) in sliding_window(image_file, stepSize=32, windowSize=(image_file.shape[1], image_file.shape[0])):
                        image_predict = []
                        
                        clone = image_file.copy()
                        
                        cv2.rectangle(clone, (x, y), (x + 190, y + 110), (255, 0, 0), 2)
                        
                        
                        detected_image = image_file[y:y+110,x:x+190]
                        detected_image = cv2.resize(detected_image, (224,224))
                        detected_image = img_to_array(detected_image)
                        detected_image = np.expand_dims(detected_image,axis=0)
                        detected_image = detected_image.astype('float32')/255
                        detected_image = np.vstack(detected_image)

                        image_predict.append(detected_image) 
                        
                        predict = model1.predict(np.array(image_predict))
                        
                        if(predict[0][1]>0.8):
                            cv2.putText(clone, "Number Plate", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (126,255,255), 2)
                        else:
                            cv2.putText(clone, "Not a Number Plate", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (126,255,255), 2)
                        
                        cv2.imshow('image',clone)
                        cv2.waitKey(1)
                        time.sleep(0.025)
                    break

KeyboardInterrupt: 

In [74]:
import eli5

In [75]:
check_image_numberplate = []

In [76]:
for i in os.listdir("/Users/syedshakeeb/Downloads/baza_slika/040603/"):
    img = cv2.imread("/Users/syedshakeeb/Downloads/baza_slika/040603/"+i)
    img = cv2.resize(img,(224,224))
    img = img_to_array(img)
    img = np.expand_dims(img,axis=0)
    check_image_numberplate.append(img)

In [77]:
check_image_numberplate = np.array(check_image_numberplate)

In [78]:
predictions = model1.predict(check_image_numberplate[9])

In [79]:
predictions

array([[0.0036222 , 0.99637777]], dtype=float32)

In [80]:
print(type(predictions), predictions.shape)

<class 'numpy.ndarray'> (1, 2)


In [135]:
top_indices = np.argsort(predictions)[0, ::-1][:2]

In [136]:
top_indices

array([1, 0])

In [105]:
from PIL import Image

In [109]:
image = Image.open("/Users/syedshakeeb/Downloads/baza_slika/040603/P1010004.jpg")
image = image.resize((224,224))
image = img_to_array(image)
image = np.expand_dims(image,axis=0)

In [113]:
from keras.applications.resnet50 import preprocess_input,decode_predictions
image = preprocess_input(image)

In [116]:
eli5.show_prediction(model1, image)

In [210]:
#class_idxs_sorted = np.argsort(y_pred.flatten())[::-1]
y_pred = model1.predict(check_image_numberplate[9])
class_idxs_sorted = np.argsort(y_pred.flatten())[::-1]

In [217]:
for i, idx in enumerate(class_idxs_sorted[:2]):
    print("Top {} predicted class:     Pr(Class={:2} ={:5.3f}".format(
          i + 1,idx,y_pred[0,idx]))

Top 1 predicted class:     Pr(Class= 0 =0.503
Top 2 predicted class:     Pr(Class= 1 =0.497


In [219]:
class_idxs_sorted

array([0, 1])

In [182]:
from vis.utils import utils
import keras

In [204]:
layer_idx = utils.find_layer_idx(model1, 'prediction')

In [205]:
model1.layers[layer_idx].activation = keras.activations.linear
model = utils.apply_modifications(model1)

In [206]:
print(model.layers)

[<keras.engine.training.Model object at 0x1ccbbf3e48>, <keras.layers.core.Dense object at 0x1cd1788438>, <keras.layers.core.Dropout object at 0x1c90a05f60>, <keras.layers.core.Dense object at 0x1cd17884e0>]


In [228]:
from vis.visualization import visualize_cam
penultimate_layer_idx = utils.find_layer_idx(model.get_layer('resnet50'), "res5c_branch2c") 
class_idx  = [0]
seed_input = image
grad_top1  = visualize_cam(model, layer_idx, class_idx, seed_input, 
                           penultimate_layer_idx = penultimate_layer_idx,#None,
                           backprop_modifier     = None,
                           grad_modifier         = None)

ValueError: `penultimate_layer_idx` needs to be before `layer_idx`

In [ ]:
def plot_map(grads):
    fig, axes = plt.subplots(1,2,figsize=(14,5))
    axes[0].imshow(_img)
    axes[1].imshow(_img)
    i = axes[1].imshow(grads,cmap="jet",alpha=0.8)
    fig.colorbar(i)
    plt.suptitle("Pr(class={}) = {:5.2f}".format(
                      classlabel[class_idx],
                      y_pred[0,class_idx]))
plot_map(grad_top1)